In [1]:
import time
%load_ext wurlitzer

WARNING 08-15 14:54:53 _custom_ops.py:14] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


/opt/vllm/lib64/python3.9/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm.commit_id'
  from vllm.version import __version__ as VLLM_VERSION


Offline inference demo
----------------------------
This is just a brief demo to show that vLLM with Spyre can be used in the offline mode. 

vLLM will determine the Spyre config automatically and warmup the Spyre stack. 
The startup of vLLM (including warmup of Spyre), is expected to take 15 min for prompt length of 64 and maximum number of decode tokens 5 (it will take ~20min for 64/20).

### 1. create vLLM instance 
(for offline usage, including warmup)

The maximum prompt length and maximum number of decode tokens can be specified using the environment variables `VLLM_SPYRE_WARMUP_PROMPT_LENS`, and `VLLM_SPYRE_WARMUP_NEW_TOKENS`. 
Otherwise the default max prompt length of 64 and maximum of 20 decode tokens will be used. 

In [2]:
import os

os.environ['VLLM_SPYRE_WARMUP_PROMPT_LENS'] = '64'
os.environ['VLLM_SPYRE_WARMUP_NEW_TOKENS'] = '5'

In [3]:
from vllm import LLM, SamplingParams

# Create an LLM.
llm = LLM(
    model="/models/llama-7b-chat",
    tokenizer="/models/llama-7b-chat",
    max_model_len=2048,
    block_size=2048,
    device="spyre")

INFO 08-15 14:54:54 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/tmp/7B-F', speculative_config=None, tokenizer='/tmp/7B-F', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/tmp/7B-F, use_v2_block_manager=False, enable_prefix_caching=False)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


WARNING 08-15 14:54:55 utils.py:581] Pin memory is not supported on Spyre device.
[SpyreWorker] environment configured
[SpyreWorker] load model...
>> DEBUG  SETUP
0 / 1 : Python Version : 3.9.18
0 / 1 : PyTorch Version: 2.2.2+cpu
0 / 1 : PCI Addr Rank 0 AIU_WORLD_RANK_0=0
0 / 1 : PCI Addr Rank 0 FLEX_RDMA_PCI_BUS_ADDR_0=0000:1e:00.0
0 / 1 : FLEX_COMPUTE=SENTIENT
0 / 1 : FLEX_DEVICE=VFIO
0 / 1 : DEEPRT_EXPORT_DIR=export/0
0 / 1 : DTCOMPILER_EXPORT_DIR=export/0
0 / 1 : AIU_CONFIG_FILE_0=/etc/aiu/senlib_config.json
0 / 1 : SENLIB_DEVEL_CONFIG_FILE=/etc/aiu/senlib_config.json
0 / 1 : FLEX_RDMA_PCI_BUS_ADDR_0=0000:1e:00.0
0 / 1 : FLEX_RDMA_LOCAL_RANK=0
0 / 1 : FLEX_RDMA_LOCAL_SIZE=1
0 / 1 : FLEX_RDMA_WORLD_RANK=0
0 / 1 : FLEX_RDMA_WORLD_SIZE=1
0 / 1 : Spyre: Enabled (0) (offset=0)
0 / 1 : Dynamo Backend  : sendnn_decoder
0 / 1 : CPU Cores       : 56 x 2 HW threads
------------------------------------------------------------
NOTICE: Adjusting torch._dynamo.config.accumulated_cache_size_limit

### 2. Create the prompt  and `SamplingParams`

In [4]:
# Sample prompts.
template = (
    "Below is an instruction that describes a task. Write a response that "
    "appropriately completes the request. Be polite in your response to the "
    "user.\n\n### Instruction:\n{}\n\n### Response:"
)
prompt1 = template.format(
    "Provide a list of instructions for preparing chicken soup for a family "
    "of four."
)
prompts = [
    prompt1,
]
print(prompts)


['Below is an instruction that describes a task. Write a response that appropriately completes the request. Be polite in your response to the user.\n\n### Instruction:\nProvide a list of instructions for preparing chicken soup for a family of four.\n\n### Response:']


In [5]:
# Create a sampling params object.
max_tokens = 5
sampling_params = SamplingParams(max_tokens=max_tokens, temperature=0.0)


### 3. Generate the response

In [6]:
print("=============== GENERATE")
t0 = time.time()
outputs = llm.generate(prompts, sampling_params)
print("Time elaspsed for %d tokens is %.2f sec" % 
      (len(outputs[0].outputs[0].token_ids), time.time()-t0))
for output in outputs:
   prompt = output.prompt
   generated_text = output.outputs[0].text
   print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
print(output.outputs[0])

=============== GENERATE


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[spyre_model_runner:execute_model] t_token: 195.92ms
[spyre_model_runner:execute_model] t_token: 158.88ms
[spyre_model_runner:execute_model] t_token: 158.49ms
[spyre_model_runner:execute_model] t_token: 158.88ms


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s, est. speed input: 76.65 toks/s, output: 5.99 toks/s]

[spyre_model_runner:execute_model] t_token: 158.67ms
Time elaspsed for 5 tokens is 0.84 sec
Prompt: 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Be polite in your response to the user.\n\n### Instruction:\nProvide a list of instructions for preparing chicken soup for a family of four.\n\n### Response:', Generated text: '\nOf course! Here'
CompletionOutput(index=0, text='\nOf course! Here', token_ids=(13, 2776, 3236, 29991, 2266), cumulative_logprob=-0.9147708129385137, logprobs=None, finish_reason=length, stop_reason=None)


Expectation (for the 2nd+ tokens): 
- ~158ms per token if the model was warmed up with max 5 output tokens
- ~162ms per token if the model was warmed up with max 20 output tokens